In [2]:
import pandas as pd
import numpy as np
import re
import time
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

import os
import sys
import urllib.request
import json
from datetime import datetime
from tqdm.notebook import tqdm

## 1. 다운로드 진행 (약 2시간 30분)
1) 수동 로그인을 한다.

2) 원하는 위치까지 클릭하여 이동한다

In [2]:
def nation_to_csv(driver, country_element):
    #start! geography 버튼 클릭
    geography_button = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[2]/span')
    try:
        geography_button.click()
    except:
        geography_button.send_keys(Keys.ENTER)
    time.sleep(1)
    
    country_element.click() #국가 결정
    time.sleep(2)
    
    apply_button = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[12]/section[2]/header/span[1]')
    apply_button.click() #apply 버튼 클릭
    time.sleep(2)
    
    try:
        csv_button = driver.find_element_by_xpath('//*[@id="modalBody"]/div[2]/div[2]/div')
        csv_button.click() #csv 버튼 클릭
    except:
        download_button = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[10]/div/div/span')
        download_button.click()
        time.sleep(0.1)
        
        csv_button_2 = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[10]/div/div/div/ul/li[2]')
        csv_button_2.click()

### 수동으로 로그인 후, 원하는 페이지까지 이동하는 작업 진행

In [3]:
url = "https://data.gsmaintelligence.com/data/market-metrics"
driver = webdriver.Chrome()
driver.get(url)

# 수동 로그인! 
# 국가 선택하기 직전까지 이동! -> 'Mobile>By Market'

### 다운로드 프로그램 실행 

In [21]:
#0.start! geography 버튼 클릭
geography_button = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[2]/span')
try:
    geography_button.click()
except:
    geography_button.send_keys(Keys.ENTER)
time.sleep(1)

region_list = driver.find_elements_by_xpath("//*[@class='link-item']") 

#2. 검색조건 세팅
#2.1. 기간 선택
last_year_dropdown = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[4]/span')
last_year_dropdown.click()
time.sleep(0.5)
five_year = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[4]/div/ul/li[5]')
five_year.click()    
time.sleep(0.5)
    
#2.2 예측 포함여부 선택
forcasting_yn = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[9]/div/label')
forcasting_yn.click()
time.sleep(0.5)

for element in region_list[237:]:
    country_name = element.text    
    #4. 다운로드
    nation_to_csv(driver, element)
    
    #5. geography 버튼 클릭
    geography_button = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[2]/span')
    try:
        geography_button.click()
    except:
        geography_button.send_keys(Keys.ENTER)
    time.sleep(1)
    
    #6. 국가 리셋
    clear_button = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/div/div[1]/div[12]/section[2]/header/span[3]')
    clear_button.click()
    try:
        geography_button.click()
    except:
        geography_button.send_keys(Keys.ENTER)
    time.sleep(10)

## 2. 다운로드 받은 파일 Excel로 변환 (약 20분)

1) 원하는 경로에 다운받은 파일들을 모두 넣는다.
    - 다른 파일은 넣으면 에러가 나므로 주의!1
2) 코드를 돌린다. 

In [ ]:
#csv파일에서 이름이 부여된 excel파일로 변환하는 작업

In [10]:
def csv_to_named_excel():    
    path_dir = 'C:\workplace\알바\카운터포인트리서치\GSMA\data'
    file_list = os.listdir(path_dir)
    
    
    for filenm in file_list[:]:
        if "xlsx" not in filenm and "csv" not in filenm:
            continue
        try:
            csv_file = pd.read_csv(path_dir+"\\"+filenm)
            csv_file.fillna("",inplace=True) #결측값 처리 => ""
        except:
            csv_file = pd.read_excel(path_dir+"\\"+filenm, skiprows =[0,1],skipfooter=3)
            csv_file.fillna("",inplace=True) #결측값 처리 => ""         
        country_len = len(csv_file.Country_name.value_counts()) #국가 Country_name
        region_len = len(csv_file.Region_name.value_counts()) #대륙(대분류) Region_name
        subregion_len = len(csv_file.Subregion_name.value_counts()) #대륙(소분류) Subregion_name

        if csv_file.Region_name.value_counts().index[0]=="World":
                final_file_nm = "World" #대륙 여러개
        elif country_len==1:
            final_file_nm = csv_file.Country_name.value_counts().index[0]
        elif subregion_len==1:
            final_file_nm = csv_file.Subregion_name.value_counts().index[0]
        elif region_len==1:
                final_file_nm = csv_file.Region_name.value_counts().index[0]

        csv_file.to_excel(path_dir+"\\excel_file\\"+final_file_nm+".xlsx", index=False)
        time.sleep(1)

In [11]:
%%time

csv_to_named_excel()

Wall time: 14min 36s
